## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等


In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

In [14]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape) #(50000, 32, 32, 3)

(50000, 32, 32, 3)


In [15]:
## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std

## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

In [16]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [ ]:
classifier=Sequential()

input_shape = (32, 32, 3)

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same', input_shape=input_shape))
classifier.add(BatchNormalization())

# MaxPooling2D 通常會在幾層 Convolution 後，用來降低 Feature Maps 的尺度以及強化特徵。
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same',input_shape=input_shape))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

# FC
# output_dim=100,activation=relu
classifier.add(Dense(units=100, activation='relu')) 

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
500/500 [==============================] - 4s 8ms/step - loss: 1.4728 - accuracy: 0.4866
Epoch 2/100
500/500 [==============================] - 4s 8ms/step - loss: 1.0613 - accuracy: 0.6289
Epoch 3/100
500/500 [==============================] - 4s 8ms/step - loss: 0.8919 - accuracy: 0.6884
Epoch 4/100
500/500 [==============================] - 4s 8ms/step - loss: 0.7840 - accuracy: 0.7246
Epoch 5/100
500/500 [==============================] - 4s 8ms/step - loss: 0.6850 - accuracy: 0.7595
Epoch 6/100
500/500 [==============================] - 4s 8ms/step - loss: 0.5913 - accuracy: 0.7934
Epoch 7/100
500/500 [==============================] - 4s 8ms/step - loss: 0.4846 - accuracy: 0.8275
Epoch 8/100
500/500 [==============================] - 4s 8ms/step - loss: 0.4128 - accuracy: 0.8575
Epoch 9/100
500/500 [==============================] - 4s 8ms/step - loss: 0.3361 - accuracy: 0.8825
Epoch 10/100
500/500 [==============================] - 4s 8ms/step - loss: 0.2776 - accura

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [ ]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.4067942e-04, 8.4954481e-05, 3.3949956e-03, 9.9373794e-01,
        8.5904851e-04, 1.4369336e-08, 1.5938883e-03, 1.6109092e-10,
        7.9198780e-06, 1.8042175e-04]], dtype=float32)

# 增加 Dropout


In [ ]:
classifier=Sequential()

input_shape = (32, 32, 3)

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

# FC
classifier.add(Dense(units=100, activation='relu', kernel_regularizer=regularizers.l2(l=0.001))) 
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(rate=0.5)) ##Dropout

classifier.add(Dense(units=100,activation='relu',kernel_regularizer=regularizers.l2(0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(rate=0.3))##Dropout

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=500)

Epoch 1/500
500/500 [==============================] - 5s 11ms/step - loss: 1.9738 - accuracy: 0.4060
Epoch 2/500
500/500 [==============================] - 5s 11ms/step - loss: 1.4887 - accuracy: 0.5562
Epoch 3/500
500/500 [==============================] - 5s 11ms/step - loss: 1.3149 - accuracy: 0.6075
Epoch 4/500
500/500 [==============================] - 5s 11ms/step - loss: 1.2046 - accuracy: 0.6389
Epoch 5/500
500/500 [==============================] - 5s 11ms/step - loss: 1.1344 - accuracy: 0.6588
Epoch 6/500
500/500 [==============================] - 5s 11ms/step - loss: 1.0837 - accuracy: 0.6759
Epoch 7/500
500/500 [==============================] - 5s 11ms/step - loss: 1.0450 - accuracy: 0.6854
Epoch 8/500
500/500 [==============================] - 5s 11ms/step - loss: 1.0243 - accuracy: 0.6965
Epoch 9/500
500/500 [==============================] - 5s 11ms/step - loss: 1.0017 - accuracy: 0.7039
Epoch 10/500
500/500 [==============================] - 5s 11ms/step - loss: 0.983

In [17]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[9.0305069e-03, 1.3905624e-04, 6.2684007e-02, 2.0848783e-02,
        6.9536191e-01, 8.6313952e-04, 1.2062297e-01, 3.2434941e-04,
        8.9965768e-02, 1.5947074e-04]], dtype=float32)